## IMPORT AND FUNCTIONS DEFINITION

In [ ]:
import os 
import numpy as np 
from math import * 
from random import randint 
import matplotlib.pyplot as plt 
from pandas import read_csv 
from statistics import mean 
from utils import * 

#ALL FUNCTIONS RETURN 0 OR 1
def neg(a): 
    if (a==1): 
         return 0 
    else: 
         return 1 

def et(a,b):  
    return min(a,b) 

def ou(a,b): 
    return max(a,b)
        
def hadamard_et(vec_a,vec_b): 
    dimension=len(vec_a) 
    result=np.zeros(dimension,dtype=np.uint32) 
    for i in range(dimension): 
         result[i]=int(et(vec_a[i],vec_b[i])) 
    return result 

def hadamard_ou(vec_a,vec_b): 
    dimension=len(vec_a) 
    result=np.zeros(dimension,dtype=np.uint32) 
    for i in range(dimension): 
         result[i]=int(ou(vec_a[i],vec_b[i])) 
    return result 

def hadamard_et_ou(vec_a,vec_b,vec_c,vec_d): 
    un=hadamard_et(vec_a,vec_c) 
    deux=hadamard_ou(vec_b,vec_d) 
    return un, deux

def atomic_cons_boolean_pair(a,b,c,d,list_valid):
    if (a,b,c,d) in list_valid:
        return 1
    return 0
    
def solve_atomic_equation(a,b,c):
    non_determ=[(0,0,0),(1,0,0),(1,0,1),(1,1,0)]
    one_sol=[(0,0,1),(0,1,0),(1,1,1)]
    l=[]
    if (a,b,c) in non_determ:
        l.append(np.random.randint(0,2))
    if (a,b,c) in one_sol:
        l.append(1)
    return l

def solve_vec_equation(vec_a,vec_b,vec_c):
    dimension=len(vec_a)
    l=np.zeros(dimension)
    for i in range(dimension):
        l1=solve_atomic_equation(vec_a[i],vec_b[i],vec_c[i])
        if len(l1)==0:
            l[i]=2
        else:
            l[i]=l1[0]
    return l
    
def hadamard_consequence_pair(vec_a,vec_b,vec_c,vec_d,list_valid): 
    dimension=len(vec_a)
    for i in range(dimension): 
        if atomic_cons_boolean_pair(vec_a[i],vec_b[i],vec_c[i],vec_d[i],list_valid)==0: 
            return 0 
    return 1

def atomic_bool_analogy(a,b,c,d,analogies): 
    #analogies=[(0,0,0,0),(1,1,1,1),(1,0,1,0),(0,1,0,1),(0,0,1,1),(1,1,0,0)] 
    if (a,b,c,d) in analogies:
        return 1
    return 0

def is_analogy_jiaf(a,b,c,d): 
    return et(atomic_consequence_for_pair(a,b,c,d),atomic_consequence_for_pair(c,d,a,b)) 

def hadamard_analogy(vec_a,vec_b,vec_c,vec_d,analogies):
    dimension=len(vec_a) 
    for i in range(dimension): 
         if atomic_bool_analogy(vec_a[i],vec_b[i],vec_c[i],vec_d[i],analogies)==0: 
            return 0 
    return 1 

def get_number_of_cons_log(a,b,list_valid):
    l=[]
    for c in [0,1]:
        for d in [0,1]:
            if atomic_cons_boolean_pair(a,b,c,d,list_valid):
                l.append((c,d))
    return l, len(l)
       
def g_6(arr): # using all attributes - all of them have the same FRI 
    if sum(arr)>=5: 
         return 1 
    return 0 

def average_creativity_from_one_pair(a,b,X): # (a,b) from X and set_of_pairs
    creativity_a_b=0
    size=X.shape[0]
    X_list=X.tolist()
    for i in range(size): #we check for every c in X
        c=X[i] # get c from X
        d=solve_vec_equation(a,b,c) # solve vector (a,b) -> (c,d)
        if not(np.in1d(np.array([2]),d)): # 2 means on one component, it does not match
            d_list=d.tolist()
            if d_list not in X_list:
                creativity_a_b+=1
    return (creativity_a_b/size) # we get average on c

def nb_cons_a_b(vec_a,vec_b,start,pairs,list_valid,analogies): #we count the total number of cons log of (a,b) in pairs
    l=len(pairs)
    list_a=sorted(vec_a.tolist())
    count_not_analogy=0
    count_analogy=0
    for i in range(start,len(pairs)):
        vec_c,vec_d = pairs[i]
        list_c=sorted(vec_c.tolist())
        if list_a != list_c: #we eliminate trivial stuff
                if (hadamard_consequence_pair(vec_a,vec_b,vec_c,vec_d,list_valid)==1): 
                    if (hadamard_analogy(vec_a,vec_b,vec_c,vec_d,analogies)==1):
                        count_analogy+=1
                    else:
                        count_not_analogy+=1
    return count_not_analogy, count_analogy
        

def testing_nb_cons(f,dimension,size,categorical_range,list_valid,analogies):
    file_created=create_categorical_dataset(f,dimension,size,categorical_range) 
    data, X, y, dimension=load_dataset(file_created) 
    pairs=all_pairs(X) 
    l=len(pairs)
    count_not_analogy=0
    count_analogy=0
    for j in range(len(pairs)): 
        vec_a,vec_b=pairs[j] #  np array
        count_not_analogy_a_b, count_analogy_a_b =nb_cons_a_b(vec_a,vec_b,0,pairs,list_valid,analogies)
        count_not_analogy+=count_not_analogy_a_b
        count_analogy+=count_analogy_a_b
        #if count_a_b > maxi:
         #   maxi=count_a_b
    #print(l,maxi)
    clean_folder("tests/")
    return count_not_analogy/l, count_analogy/l

def creativity(f,dimension,size,categorical_range):
    file_created=create_categorical_dataset(f,dimension,size,categorical_range) 
    data, X, y, dimension=load_dataset(file_created) 
    pairs=all_pairs(X) 
    l=len(pairs)
    i=np.random.randint(0,l) #we choose a pair
    vec_a,vec_b=pairs[i]
    #j=np.random.randint(0,X.shape[0])
    count_old=0
    count_new=0
    for i in range(X.shape[0]):
        vec_c=X[i]
        vec_d=solve_vec_equation(vec_a,vec_b,vec_c)
        if not(np.in1d(np.array([2]),vec_d)):
            X_list=X.tolist()
            vec_d_list=vec_d.tolist()
            if vec_d_list in X_list:
                #print("in X",vec_d)
                count_old+=1
            else:
                #print("Not in X",vec_d)
                count_new+=1
    clean_folder("tests/")
    return count_old, count_new

def creativity_all_new(f,dimension,size,categorical_range):
    file_created=create_categorical_dataset(f,dimension,size,categorical_range) 
    data, X, y, dimension=load_dataset(file_created) 
    pairs=all_pairs(X) 
    l=len(pairs)
    res=0
    list_of_values=[]
    for j in range(l):
        vec_a,vec_b=pairs[j]        # get a pair (a,b)
        n=average_creativity_from_one_pair(vec_a,vec_b,X) #average number of new d wrt c
        res+=n
        list_of_values.append(n)
    clean_folder("tests/")
    return res/l, np.std(np.asarray(list_of_values))
        

def creativity_all(f,dimension,size,categorical_range):
    file_created=create_categorical_dataset(f,dimension,size,categorical_range) 
    data, X, y, dimension=load_dataset(file_created) 
    X_list=X.tolist()
    pairs=all_pairs(X) 
    l=len(pairs)
    total_list_number_in=[]
    total_list_number_out=[]
    for j in range(l):
        vec_a,vec_b=pairs[j]        # get a pair (a,b)
        count_in_a_b=0
        count_out_a_b=0
        for i in range(X.shape[0]): # get a vector c
            vec_c=X[i]
            vec_d=solve_vec_equation(vec_a,vec_b,vec_c) # solve vector (a,b) -> (c,d)
            if not(np.in1d(np.array([2]),vec_d)):       # 2 means on one component, it does not match
                vec_d_list=vec_d.tolist()
                if vec_d_list in X_list: # d in X
                    count_in_a_b+=1
                else:                    # d not in X
                    count_out_a_b+=1
        total_list_number_in.append(count_in_a_b/size)    # average number of d in  wrt c
        total_list_number_out.append(count_out_a_b/size)  # average number of d out wrt c
    a=sum(np.asarray(total_list_number_in))/l
    b=sum(np.asarray(total_list_number_out))/l
    a_std=np.std(np.asarray(total_list_number_in))
    b_std=np.std(np.asarray(total_list_number_out))
    clean_folder("tests/")
    return a,a_std,b,b_std

def creativity_et_ou(f,dimension,size,categorical_range):
    file_created=create_categorical_dataset(f,dimension,size,categorical_range) 
    data, X, y, dimension=load_dataset(file_created) 
    pairs=all_pairs(X) 
    #print(type(X))
    l=len(pairs)
    #print(l,X.shape)
    count_new=0
    X_list=X.tolist()
    full_list=[]
    #print(X_list)
    for i in range(len(pairs)):
        vec_a,vec_b=pairs[i]
        for vec_c,vec_d in pairs:
            un,deux= hadamard_et_ou(vec_a,vec_b,vec_c,vec_d)
            if (un.tolist() in X_list) or (un.tolist() in full_list):
                #print(un.tolist())
                continue
            else:
                full_list.append(un.tolist())
                count_new+=1 
            if (deux.tolist() in X_list) or (deux.tolist() in full_list):
                #print(un.tolist())
                continue
            else:
                full_list.append(deux.tolist())
                count_new+=1           
    clean_folder("tests/")
    #print("job done:",count_new)
    return count_new



## ACTION AND TESTS

In [ ]:
#GLOBAL VARIABLES
f=g_6  #USELESS AT THIS STAGE
list_valid=[(0,0,0,0),(0,0,0,1),(0,0,1,1),(0,1,0,1),(1,0,0,0),(1,0,0,1),(1,0,1,0),(1,0,1,1),(1,1,0,0),(1,1,0,1),(1,1,1,1)]
analogies=[(0,0,0,0),(1,1,1,1),(1,0,1,0),(0,1,0,1),(0,0,1,1),(1,1,0,0)]
dimension=50 
size=100
categorical_range=1 
number_of_test=10
count_old=0
count_new=0
count_not_analogy,count_analogy=0,0
list_count_not_analogy=[]
list_count_analogy=[]

#for a in [0,1]:
#    for b in [0,1]:
#        for c in [0,1]:
#            for d in [0,1]:
#                print(a,b,c,d,atomic_cons_boolean_pair(a,b,c,d,list_valid),atomic_bool_analogy(a,b,c,d,analogies))
                
#vec_a=np.array([0,1,1,1,1])
#vec_b=np.array([0,0,1,1,1])
#vec_c=np.array([0,0,1,1,0])
#vec_d=np.array([1,0,0,1,0])
#print(hadamard_consequence_pair(vec_a,vec_b,vec_c,vec_d,list_valid),hadamard_analogy(vec_a,vec_b,vec_c,vec_d,analogies))

#LOGICAL CONSEQUENCES INSIDE
#for i in range(number_of_test):
#    no, yes =testing_nb_cons(f,dimension,size,categorical_range,list_valid,analogies)
#    #print(no,yes)
#    list_count_not_analogy.append(no)
#    list_count_analogy.append(yes)
#    count_not_analogy+=no
#    count_analogy+=yes
#print(list_count_not_analogy)
#std_list_count_not_analogy = np.std(np.asarray(list_count_not_analogy))
#std_list_count_analogy = np.std(np.asarray(list_count_analogy))
#print("average cons log:",int(count_not_analogy/number_of_test), "std dev:",std_list_count_not_analogy,"average analogy:", int(count_analogy/number_of_test),"std dev:",std_list_count_analogy)

#CREATIVITY ET/OU
list_values=[]
for i in range(number_of_test):
    c=creativity_et_ou(f,dimension,size,categorical_range)
    count_new+=c
    list_values.append(c)
print(list_values)
print(count_new/number_of_test, np.std(np.asarray(list_values)))

#for a in [0,1]:
#    for b in [0,1]:
#        l,n=get_number_of_cons_log(a,b,list_valid)
#        print(a,b,l,n)

#CREATIVITY ALL
#list_values_a=0
#list_values_astd=0

#for i in range(number_of_test):
#    n,astd=creativity_all_new(f,dimension,size,categorical_range)
#    #print(a,astd)
#    list_values_a+=n
#    list_values_astd+=astd
    
#print(list_values_a/number_of_test,list_values_astd/number_of_test)
